<a href="https://colab.research.google.com/github/MeGaNeKoS/Google_Colab/blob/master/GDrive_Direct_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDrive Downloader 

In [ ]:
#@title Mount Google Drive
#@markdown To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive')

## Start Download

###Torrent

In [ ]:
#@title Adding files
try:
  import libtorrent as lt
except:
  !apt install python3-libtorrent
  import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

path = "/content/drive/Shared drives/Anime/Torrent" #@param {type:"string"}
Torrent = False #@param {type:"boolean"}

def duration(num,msg):
  if int((num)//3600)>0:
    return f"{msg}{int((num)//3600)} hour : {int((num)//60)%60} min : {int((num)%60)} sec"
  elif int((num)//60)>0:
    return f"{msg}{int((num)//60)} min : {int((num)%60)} sec"
  else:
    return f"{msg}{int((num)%60)} sec"
if Torrent:
  print("Stop the code or cancel upload when you finish add the torrent files")
while True:
  if Torrent:
    from google.colab import files
    source = files.upload()
    try:
      params = {
          "save_path": path,
          "ti": lt.torrent_info(list(source.keys())[0]),
      }
      downloads.append(ses.add_torrent(params))
    except:
      break
  else:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, {"save_path": path})
    )


In [ ]:
#@title Display download progress
import time
from IPython.display import display
import ipywidgets as widgets
import threading

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]

print("You can stop, add files then resume again. this is only for displaying progress")
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
            try:
              x = round(((s.total_done/((s.progress * 100)/100)) - s.total_done)/s.download_rate)
              if x > 359999:
                filetime = duration(359999,'')
              else:
                filetime = duration(x,'')
            except:
              filetime = duration(359999,'')

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                    filetime,
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

def done():
  from google.colab import output
  output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')

print("done")
t1 = threading.Thread(target=done, args=(), daemon = True)
t1.start()
time.sleep(5)
if t1.is_alive():
  print("\nYOU CAN IGNORE ANYTHING BELOW THIS.\nTHIS IS JUST TO FORCE STOP THE PROGRAM DUE AUDIO NOTIFICATION ERROR\n\n")
  raise SystemExit()  #sometime it got stuck when playing the notif audio.